<a href="https://colab.research.google.com/github/DocBader/NumeraiAllstars/blob/main/NMRSKIZZE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Step 1.Importing Libraries**

In [ ]:
# hier fügen wir nach und nach libs hinzu

In [ ]:
import pandas as pd
import numpy as np

**Step 2. Loading the Data**

In [ ]:
df_train = pd.read_csv("numerai_training_data.csv").set_index("id")
df_test = pd.read_csv('numerai_tournament_data.csv').set_index('id')

#hier noch validation dataset und in diesem step kann man nach und nach ergänzen was für nen type of data man konkret bearbeiten will

**Step 2.5 Easier written code for some tasks**

In [ ]:
feature_cols_to_keep = [] #welche features verwenden wir beim training
eras_to_exclude_from_trainingset = []
kept_eras_in_trainingset = []
save_good_models  = True
save_result_file = True
cv_number = [] #anzahl an cross validations




**Code for removing all the unnecessary columns**

In [ ]:
no_feature_cols = ['id', 'era', 'data_type', 'target']

**Step 3. Splitting Data into X and Y**

X sind wie in den meisten ML Projekten die Features und Y die Target Variable

In [ ]:
X = df_train[features].values
y = df_train.target.values
features = [f for f in df_train if f.startswith('feature')]



In [ ]:
from sklearn.model_selection import train_test_split




X_train, X_test, y_train, y_test = train_test_split ( X, y, test_size = 0.5, train_size = 0.5)

Hier beginnt der Salat. Wir müssen herausfinden warum wenn wir das Dataset splitten, der GridSearchCV part nicht klappt.

**Step 4. GridsearchCV for Parameter Tuning**

In [ ]:
from sklearn.model_selection import GridSearchCV
 
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
  



# hier beim GridSearch gehen wir erstmal die basic algos von sklearn durch. Für den Anfagn würde das reichen um sich ein BIld zu machen ob eins davon wenigsten ne etwas gute Accuracy hat

In [ ]:
model_params = {
    'svm' :{
        'model' : svm.SVC(gamma='auto'),
        'params' : {
            'C' : [1,10,20],
            'kernel' : ['rbf', 'linear']
        }


    },

    'random_forest' :{
        'model' : RandomForestClassifier(),
        'params': {
            'n_estimators': [1,5,10]
        }

    },

    'logistic_regression' : {
        'model' : LogisticRegression(solver = 'liblinear', multi_class='auto'),
        'params' :{
            'C':[1,5,10]
        }
    },

    'naive_bayes_gaussian' : {
        'model' : GaussianNB(),
        'params' :{}

    },

    'naive_bayes_multinomial' :{
        'model' : MultinomialNB(),
        'params' : {}
    },

    'decision_treee_classifier' :{
        'model' : DecisionTreeClassifier(),
        'params' : {
            'criterion' : ['gini', 'entropy']
        } 
    }
}

In [ ]:
from sklearn.model_selection import GridSearchCV
scores = []

for model_name, mp in model_params.items():
  clf = GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
  clf.fit(X_train,y_train)
  scores.append({
      'model': model_name,
      'best_score' : clf.best_score_,
      'best_params' : clf.best_params_
  })

Der ganze GridSearch Code funktioniert in der Regel so bei den meisten anderen Projekten. Die Fehler meldung die bei mir grade kam war 'Input contains NaN, infinity or a value too large for dtype('float64').' das heisst erstmal daten halbieren oder verändern... evtl wirklich daten in hälfte teilen weil zu viel input...

**Step 5. Creating Preliminary XGBOOST**

In [ ]:
model = XGBRegressor(max_depth=5, learning_rate=0.01, n_estimators=2000, n_jobs= -1, colsample_bytree=0.1)


model.fit(X,y)

Das XGBoost model können wir mithilfe der Ergebnisse im gridsearch verfeinern. also unsere mission ist erstmal vernfüfntig gridsearch auszuführen. Probier bitte aus evtl guck ob du andere Wege findest mit den Daten zu hantieren. 

**Zusatz Optionen die erfolgreich sein könnten wenn wir die Daten richtig einsetzen : SVM**

**Preprocessing the Data**

In [ ]:
from sklearn import preprocessing, pipeline

preprocessor = pipeline.Pipeline(
    [
     ('ss', preprocessing.StandardScaler())
    ]
)

preprocessor.fit(X_training)

X_training = preprocessor.transform(X_training)

In [ ]:

X = df_train.drop(no_feature_cols, axis=1)


y = df_train['target']


X_train, X_test, y_train, y_test = train_test_split(X_training,y,test_size = 0.5, random_state=42)


X_training = X.values #creating numpy arrays

y_training = y.values


In [ ]:

from sklearn import model_selection

X_train, X_test, y_train, y_test = model_selection.train_test_split(
    X_training,
    y_training,
    test_size=0.5,
    random_state=42,

from sklearn import ensemble

model = ensemble.ExtraTreesClassifier(
    n_estimators=10,
    max_depth=3,
    random_state=42,
    n_jobs=-1,
)

from sklearn import model_selection

X_train, X_test, y_train, y_test = model_selec